<a href="https://colab.research.google.com/github/JangLaika/create-google-workspace-customer/blob/main/google_workspace_reseller_end_to_end_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Workspace Reseller: End-to-End Customer Provisioning (Colab)

> 구성: Reseller API + Site Verification API + Admin SDK Directory API

이 노트북은 다음 Codelab 내용을 **Colab** 환경에서 바로 실행할 수 있게 정리했습니다.

- 단계 요약 ([공식 문서): 고객 프로비저닝 흐름](https://developers.google.com/workspace/admin/reseller/v1/codelab/end-to-end)
  1. **Site Verification API**로 도메인 인증 토큰 생성 (DNS/META)
  2. **Reseller API**로 **Customer** 생성
  3. **Directory API**로 첫 사용자 생성 후 **Super Admin** 권한 부여
  4. **Reseller API**로 **Subscription** 생성
  5. **Site Verification API**로 최종 도메인 인증

⚠️ **전제 조건**
- Google Workspace **리셀러** 계정 & 파트너 계약
- Google Cloud 프로젝트에서 **Reseller API**, **Site Verification API**, **Admin SDK** 사용 설정
- **도메인 전체 위임(Domain-wide Delegation)**이 활성화된 **서비스 계정 키(JSON)**
- 리셀러 슈퍼관리자 이메일 (위임 대상)

실행 전, 각 셀의 **환경 변수**를 본인 값으로 바꿔주세요.

In [ ]:
# 1) 라이브러리 설치
!pip -q install google-api-python-client google-auth google-auth-httplib2 google-auth-oauthlib

In [ ]:
from google.colab import files
import json

print("서비스 계정 JSON 키 파일(.json)을 업로드하세요…")
uploaded = files.upload()

# 업로드된 파일 중 .json 우선 선택
json_files = [name for name in uploaded.keys() if name.lower().endswith(".json")]
if not json_files:
    raise ValueError("❌ 업로드된 파일 중 JSON(.json)이 없습니다. 서비스 계정 키 JSON을 업로드하세요.")

SERVICE_ACCOUNT_FILE = json_files[0]
print("업로드됨:", SERVICE_ACCOUNT_FILE)

# JSON 형식/필수키 검증
with open(SERVICE_ACCOUNT_FILE, "r", encoding="utf-8") as f:
    data = json.load(f)

required = {"type", "private_key", "client_email", "token_uri"}
missing = required - set(data.keys())
if missing or data.get("type") != "service_account":
    raise ValueError(
        f"❌ 서비스 계정 JSON 형식 오류. 누락 키: {missing} "
        f"또는 type이 'service_account'가 아님.\n"
        f"→ Cloud Console에서 서비스 계정 **JSON 키**를 다시 발급해 업로드하세요."
    )

print("✅ 서비스 계정 JSON 검증 완료:", data["client_email"])

서비스 계정 JSON 키 파일(.json)을 업로드하세요…


Saving sys-21152533756596313446997224-ebb1ddfede05.json to sys-21152533756596313446997224-ebb1ddfede05.json
업로드됨: sys-21152533756596313446997224-ebb1ddfede05.json
✅ 서비스 계정 JSON 검증 완료: creategoogleworkspacecustomern@sys-21152533756596313446997224.iam.gserviceaccount.com


In [ ]:
# 3) 구성 변수 설정 — 반드시 본인 환경에 맞게 변경하세요.
DELEGATED_ADMIN_EMAIL = 'laika.jang@netkillersoft.com'  # 리셀러 슈퍼관리자(위임 대상)

# --- 필수: 대체 이메일을 '이메일 형식'으로 수정 ---
ALTERNATE_EMAIL = 'laika.jang@netkillersoft.com'  # 고객 담당자 메일(연락용)
# 신규 고객 도메인 & 기본 정보
CUSTOMER_DOMAIN = 'laikacreate4.betanetkillersoft.com'
# --- 필수: contactName 추가 ---
POSTAL_ADDRESS = {
    'contactName': 'Laika Jang',             # ← 필수: 담당자명
    'organizationName': 'New Customer Inc2.',
    'postalCode': '04524',
    'region': 'Seoul',                        # 시/도
    'countryCode': 'KR',
    'locality': 'Jung-gu',                    # 시/군/구
    'addressLine1': '123 Example-ro'
}

# (선택) 전화번호는 국제 형식 권장. 예: +82-2-0000-0000
CUSTOMER_PHONE = '+82-2-0000-0000'

# 첫 관리자 계정
PRIMARY_ADMIN_GIVEN_NAME = 'First'
PRIMARY_ADMIN_FAMILY_NAME = 'Admin'
PRIMARY_ADMIN_EMAIL = 'admin@' + CUSTOMER_DOMAIN
PRIMARY_ADMIN_PASSWORD = 'TempPass!234'  # 초기 임시 비밀번호 정책에 맞게 설정

# 구독(SKU/플랜) 설정 예시 (Business Starter Flexible)
SKU_ID = '1010020027'  # 필요 시 다른 SKU로 변경
SEAT_NUMBER = 1  # 좌석수
PLAN = 'TRIAL'  # 'FLEXIBLE' | 'ANNUAL_MONTHLY_PAY' | 'ANNUAL_YEARLY_PAY'
PURCHASE_ORDER_ID = 'PO-' + datetime.utcnow().strftime('%Y%m%d%H%M%S')



/tmp/ipython-input-1323693637.py:34: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  PURCHASE_ORDER_ID = 'PO-' + datetime.utcnow().strftime('%Y%m%d%H%M%S')


In [ ]:
# 4) 인증 & 클라이언트 생성
from google.oauth2 import service_account
from googleapiclient.discovery import build

SCOPES = [
    'https://www.googleapis.com/auth/apps.order',      # Reseller API
    'https://www.googleapis.com/auth/siteverification',# Site Verification API (관리)
    'https://www.googleapis.com/auth/admin.directory.user', # Directory API (User 관리)
    'https://www.googleapis.com/auth/admin.directory.user.security' # makeAdmin 등에 필요
]

creds = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES
).with_subject(DELEGATED_ADMIN_EMAIL)  # 도메인 전체 위임

reseller = build('reseller', 'v1', credentials=creds)
sitever = build('siteVerification', 'v1', credentials=creds)
directory = build('admin', 'directory_v1', credentials=creds)

print('클라이언트 준비 완료')

클라이언트 준비 완료


In [ ]:
# 5) (선행) Site Verification 토큰 생성 — DNS 또는 META 방식
method = 'DNS_TXT'  # 'DNS_TXT' 또는 'META'
token_req = {
    'verificationMethod': 'DNS_TXT' if method == 'DNS_TXT' else 'META',
    'site': {'identifier': CUSTOMER_DOMAIN, 'type': 'INET_DOMAIN'}
}
token_res = sitever.webResource().getToken(body=token_req).execute()
token = token_res.get('token')
print('검증 토큰:', token)
if method == 'DNS_TXT':
    print(f"\nDNS TXT 레코드를 추가하세요:\n  호스트: @ (또는 도메인 루트)\n  값: {token}\n  TTL: 300s (권장)")
else:
    print(f"\n웹 사이트 루트에 META 태그 추가:\n  <meta name=\"google-site-verification\" content=\"{token}\">\n")
print('\n토큰 배치 후, 아래 최종 인증 셀을 실행하세요.')

검증 토큰: google-site-verification=HDQxfD_wCwh6fBpLWgHgdY2t1Hb8XsAH3mvXQdPyn_w

DNS TXT 레코드를 추가하세요:
  호스트: @ (또는 도메인 루트)
  값: google-site-verification=HDQxfD_wCwh6fBpLWgHgdY2t1Hb8XsAH3mvXQdPyn_w
  TTL: 300s (권장)

토큰 배치 후, 아래 최종 인증 셀을 실행하세요.


In [ ]:
# # 6) Reseller API: Customer 생성
# customer_body = {
#     'customerDomain': CUSTOMER_DOMAIN,
#     'alternateEmail': ALTERNATE_EMAIL,
#     'phoneNumber': CUSTOMER_PHONE,
#     'postalAddress': POSTAL_ADDRESS
# }
# try:
#     new_customer = reseller.customers().insert(body=customer_body).execute()
#     print('고객 생성 완료:\n', json.dumps(new_customer, indent=2, ensure_ascii=False))
#     CUSTOMER_ID = new_customer.get('customerId')
# except Exception as e:
#     print('고객 생성 중 오류:', e)
#     CUSTOMER_ID = None

# --- 고객 생성 재시도 ---
customer_body = {
    'customerDomain': CUSTOMER_DOMAIN,
    'alternateEmail': ALTERNATE_EMAIL,
    'phoneNumber': CUSTOMER_PHONE,
    'postalAddress': POSTAL_ADDRESS
}

try:
    new_customer = reseller.customers().insert(body=customer_body).execute()
    print('고객 생성 완료:\n', json.dumps(new_customer, indent=2, ensure_ascii=False))
    CUSTOMER_ID = new_customer.get('customerId')
except Exception as e:
    print('고객 생성 중 오류:', e)
    CUSTOMER_ID = None

고객 생성 중 오류: <HttpError 409 when requesting https://reseller.googleapis.com/apps/reseller/v1/customers?alt=json returned "Resource already exists". Details: "[{'message': 'Resource already exists', 'domain': 'global', 'reason': 'duplicate'}]">


In [ ]:
# 7) Directory API: 첫 사용자 생성 & 관리자 승격
user_body = {
    'primaryEmail': PRIMARY_ADMIN_EMAIL,
    'name': {
        'givenName': PRIMARY_ADMIN_GIVEN_NAME,
        'familyName': PRIMARY_ADMIN_FAMILY_NAME
    },
    'password': PRIMARY_ADMIN_PASSWORD
}
try:
    user = directory.users().insert(body=user_body).execute()
    print('사용자 생성 완료:', user.get('primaryEmail'))
    directory.users().makeAdmin(userKey=PRIMARY_ADMIN_EMAIL, body={'status': True}).execute()
    print('관리자 권한 부여 완료')
except Exception as e:
    print('사용자 생성/관리자 부여 중 오류:', e)

사용자 생성 완료: admin@laikacreate4.betanetkillersoft.com
관리자 권한 부여 완료


In [ ]:
# 8) Reseller API: 구독 생성
if not 'CUSTOMER_ID' in globals() or not CUSTOMER_ID:
    print('CUSTOMER_ID 가 없어 구독 생성을 건너뜁니다. 고객 생성 셀을 먼저 성공시켜주세요.')
else:
    subscription_body = {
        'customerId': CUSTOMER_ID,
        'skuId': SKU_ID,
        'plan': {'planName': PLAN},
        'purchaseOrderId': PURCHASE_ORDER_ID,
        'seats': {
            'numberOfSeats': SEAT_NUMBER
        }
    }
    try:
        sub = reseller.subscriptions().insert(customerId=CUSTOMER_ID, body=subscription_body).execute()
        print('구독 생성 완료:\n', json.dumps(sub, indent=2, ensure_ascii=False))
        SUBSCRIPTION_ID = sub.get('subscriptionId')
    except Exception as e:
        print('구독 생성 중 오류:', e)

구독 생성 중 오류: <HttpError 400 when requesting https://reseller.googleapis.com/apps/reseller/v1/customers/C01fgb0ie/subscriptions?alt=json returned "The seats provided are not valid". Details: "[{'message': 'The seats provided are not valid', 'domain': 'global', 'reason': 'invalid'}]">


In [ ]:
# 고객 생성 시 출력됐던 ID를 그대로 할당
CUSTOMER_ID = "C01fgb0ie"

print("CUSTOMER_ID =", CUSTOMER_ID)

# 9) Site Verification: 최종 인증 (토큰 배치 이후 실행)
try:
    insert_req = {
        'site': {'identifier': CUSTOMER_DOMAIN, 'type': 'INET_DOMAIN'}
    }
    verified = sitever.webResource().insert(verificationMethod='DNS_TXT', body=insert_req).execute()
    print('도메인 인증 완료:\n', json.dumps(verified, indent=2, ensure_ascii=False))
except Exception as e:
    print('도메인 인증 중 오류:', e)
    print('DNS 전파가 아직 끝나지 않았거나 토큰 배치가 누락되었을 수 있습니다.')

CUSTOMER_ID = C01fgb0ie
도메인 인증 중 오류: <HttpError 400 when requesting https://www.googleapis.com/siteVerification/v1/webResource?verificationMethod=DNS_TXT&alt=json returned "The necessary verification token could not be found on your site.". Details: "[{'message': 'The necessary verification token could not be found on your site.', 'domain': 'global', 'reason': 'badRequest'}]">
DNS 전파가 아직 끝나지 않았거나 토큰 배치가 누락되었을 수 있습니다.


## 참고 및 체크리스트
- Cloud Console에서 API 사용 설정: **Reseller, Site Verification, Admin SDK**
- 서비스 계정: **도메인 전체 위임** 활성화, **OAuth 범위** 승인
- Reseller API 권한은 **리셀러 도메인** 기준으로 동작합니다.
- Site Verification **토큰 배치** 후, DNS 전파 지연(수분~수시간) 가능
- 구독 SKU/플랜/좌석 수는 실제 계약/정책에 맞게 조정하세요.

문제 해결은 각 API의 오류 메시지와 공식 문서를 참고하세요.

1) 고객 상태 확인 (verified 플래그 체크)

customerDomainVerified 가 true 로 보이면 인증 상태가 완전히 반영된 것입니다. (전파 중이면 잠시 false일 수 있어요.)

In [ ]:
cust = reseller.customers().get(customerId=CUSTOMER_ID).execute()
print("customerDomainVerified =", cust.get("customerDomainVerified"))
print(json.dumps(cust, indent=2, ensure_ascii=False))

customerDomainVerified = False
{
  "kind": "reseller#customer",
  "customerId": "C01fgb0ie",
  "customerDomain": "laikacreate4.betanetkillersoft.com",
  "postalAddress": {
    "kind": "customers#address",
    "contactName": "Laika Jang",
    "organizationName": "New Customer Inc2.",
    "locality": "Jung-gu",
    "region": "Seoul",
    "postalCode": "04524",
    "countryCode": "KR",
    "addressLine1": "123 Example-ro"
  },
  "alternateEmail": "laika.jang@netkillersoft.com",
  "resourceUiUrl": "https://admin.google.com/ac/home?ecid=C01fgb0ie",
  "customerDomainVerified": false,
  "customerType": "domain"
}


2) 구독 생성 (Business Starter / FLEXIBLE 예시)

흔한 오류 팁

400 invalid sku: SKU_ID 문자열이 계약/리전과 안 맞는 경우가 많아요.
→ 리셀러 계약서/콘솔에서 가능한 SKU를 확인해 정확한 ID로 넣어주세요. (e.g. Business Standard, Business Plus 등)

seats/plan 정책 오류: ANNUAL 플랜은 최소 좌석수/기간 조건이 있어요. 테스트는 FLEXIBLE이 편합니다.

In [ ]:
# 정확한 SKU ID (Business Starter)
SKU_ID = '1010020027'

SEAT_NUMBER = 1  # 체험 최대 좌석(예시). 필요에 맞게 조정
PLAN = 'TRIAL'    # 30일 체험

from datetime import datetime, timezone
PURCHASE_ORDER_ID = 'PO-' + datetime.now(timezone.utc).strftime('%Y%m%d%H%M%S')

subscription_body = {
    'customerId': CUSTOMER_ID,
    'skuId': SKU_ID,
    'plan': {'planName': PLAN},
    'purchaseOrderId': PURCHASE_ORDER_ID,
    'seats': {
        'maximumNumberOfSeats': SEAT_NUMBER  # TRIAL/FLEXIBLE은 maximumNumberOfSeats 사용
    }
}

try:
    sub = reseller.subscriptions().insert(customerId=CUSTOMER_ID, body=subscription_body).execute()
    print('구독 생성 완료:\n', json.dumps(sub, indent=2, ensure_ascii=False))
    SUBSCRIPTION_ID = sub.get('subscriptionId')
except Exception as e:
    print('구독 생성 중 오류:', e)


구독 생성 완료:
 {
  "kind": "reseller#subscription",
  "customerId": "C01fgb0ie",
  "subscriptionId": "9952488829",
  "skuId": "1010020027",
  "creationTime": "1761203856956",
  "billingMethod": "ONLINE",
  "plan": {
    "planName": "TRIAL",
    "isCommitmentPlan": false
  },
  "seats": {
    "kind": "subscriptions#seats",
    "licensedNumberOfSeats": 0,
    "maximumNumberOfSeats": 1
  },
  "trialSettings": {
    "isInTrial": true,
    "trialEndTime": "1763799456847"
  },
  "purchaseOrderId": "PO-20251023071733",
  "status": "SUSPENDED",
  "suspensionReasons": [
    "PENDING_TOS_ACCEPTANCE"
  ],
  "resourceUiUrl": "https://admin.google.com/ac/billing/subscriptions?ecid=C01fgb0ie",
  "customerDomain": "laikacreate4.betanetkillersoft.com",
  "skuName": "Google Workspace Business Starter"
}
